Get Started With LangChain
simple LLM Calls with streaming
Dynamic prompt temmplates (translate app)
Building chains (story generator with analysis)
Conversational Q&A Assitant with memory
Tool integration (calculator & weather)

In [48]:
import langchain
import os
from dotenv import load_dotenv
load_dotenv()

True

In [49]:
os.environ["OPENAI_API_KEY"]= os.getenv("OPENAI_API_KEY")

### Example: Simple LLM Call With streaming

In [50]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage


In [51]:
model = init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x112276c40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1122775c0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

Another way to create model

In [52]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x112cb8180>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x112cb8b00>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

<h4> Type of message </h4>
human message is the input to the llm <br>
ouput message is ai message ie. return result <br>
system message: it's an instruction to the LLM how to behave <br>


In [53]:
## create messages
messages = [
    SystemMessage("You're helpful ai agent"),
    HumanMessage("What are the top 3 benefits of using langchain")
    
]



In [54]:
response = model.invoke(messages)
response

AIMessage(content="Langchain is an open-source platform that enables users to build, deploy, and manage AI models, particularly those based on large language models (LLMs). Based on its capabilities and applications, here are three potential benefits of using Langchain:\n\n1. **Efficient Model Development and Deployment**: Langchain simplifies the process of developing and deploying AI models by providing a unified platform for tasks such as model training, testing, and deployment. This streamlined workflow enables developers to focus on more complex aspects of model development, accelerating the development and deployment of AI applications.\n\n2. **Enhanced Model Customization and Control**: Langchain offers a range of tools and libraries that allow users to customize and fine-tune AI models to suit their specific needs. This includes the ability to specify model architectures, training data, and hyperparameters, enabling developers to create models that are optimized for their parti

In [55]:
print(response.content)

Langchain is an open-source platform that enables users to build, deploy, and manage AI models, particularly those based on large language models (LLMs). Based on its capabilities and applications, here are three potential benefits of using Langchain:

1. **Efficient Model Development and Deployment**: Langchain simplifies the process of developing and deploying AI models by providing a unified platform for tasks such as model training, testing, and deployment. This streamlined workflow enables developers to focus on more complex aspects of model development, accelerating the development and deployment of AI applications.

2. **Enhanced Model Customization and Control**: Langchain offers a range of tools and libraries that allow users to customize and fine-tune AI models to suit their specific needs. This includes the ability to specify model architectures, training data, and hyperparameters, enabling developers to create models that are optimized for their particular use cases.

3. **

In [56]:
model.invoke([HumanMessage("What is machine learning")])

AIMessage(content='**Machine Learning: An Overview**\n\nMachine learning is a subfield of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance over time. The primary goal of machine learning is to develop systems that can automatically improve their performance on a task without being explicitly programmed.\n\n**Key Characteristics of Machine Learning:**\n\n1. **Learning from data**: Machine learning algorithms are trained on a dataset, which allows them to learn patterns, relationships, and decision boundaries.\n2. **Improving performance**: As the algorithm is trained on more data or experiences, its performance improves over time.\n3. **Autonomous decision-making**: Machine learning systems can make decisions without human intervention, based on the patterns and relationships learned from the data.\n\n**Types of Machine Learning:**\n\n1. **Supervised Learning**: Th

In [57]:
## Streaming example
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

Langchain is an AI platform that enables users to build and deploy conversational AI models. Here are the top 3 benefits of using Langchain:

1. **Conversational AI Model Development**: Langchain provides a user-friendly interface and a range of tools to help users develop and deploy conversational AI models. With Langchain, users can create custom models that can engage in natural-sounding conversations, answer complex questions, and perform a variety of tasks.

2. **Integration with LLMs (Large Language Models)**: Langchain is built on top of large language models (LLMs) such as Llama and other models. This integration enables users to tap into the capabilities of these powerful models and leverage their language understanding and generation capabilities.

3. **Access to a Wide Range of Model Customization Options**: Langchain allows users to customize their models to suit specific use cases. This includes the ability to fine-tune models on custom datasets, integrate external knowled

## Dynamic Prompt templates


In [58]:
from langchain_core.prompts import ChatPromptTemplate

### create translation app

translation_template = ChatPromptTemplate.from_messages([
    ("system", "You're a professional translate. Translate the following text {text} from {source_language} to {target_language}. Main the tone and style"),
    ("human", "{text}")
])
## using the template
prompt = translation_template.invoke({
    "source_language": "english",
    "target_language": "french",
    "text": "langchain makes ai application incredible easy !"
})

In [59]:
prompt

ChatPromptValue(messages=[SystemMessage(content="You're a professional translate. Translate the following text langchain makes ai application incredible easy ! from english to french. Main the tone and style", additional_kwargs={}, response_metadata={}), HumanMessage(content='langchain makes ai application incredible easy !', additional_kwargs={}, response_metadata={})])

In [60]:
response = model.invoke(prompt)

In [61]:
print(response.content)

Langchain rend les applications AI incroyablement faciles à utiliser !


### Building your first chain

In [70]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def createStoryChain():

    #Template for story generation

    story_prompt = ChatPromptTemplate(
        [("system", "you're a creative storyteller. right a short and enganging story based on the given theme, character and setting"),
        ("user", "Theme: {theme}\n Main character: {character} \n setting: {setting}")]
    )

    # template for story analysis
    analysis_prompt = ChatPromptTemplate(
       [("system", "you're a literary critic. analyze the following story and insights."),
        ("user", "{story}")]
    )

    story_chain = (
        story_prompt| model | StrOutputParser()
    )

    #create a function to pass the story to analysis
    def analyze_story(story_text):
        return {"story": story_text}

    analysis_chain = (
        story_chain
        | RunnableLambda(analyze_story)
        | analysis_prompt
        | model
        | StrOutputParser()
    )

    return analysis_chain

In [71]:
chain = createStoryChain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="you're a creative storyteller. right a short and enganging story based on the given theme, character and setting"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\n Main character: {character} \n setting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x112cb8180>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x112cb8b00>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_types={}, par

In [75]:
result = chain.invoke({
    "theme": "artifical intelligence",
    "character": "a curious robot",
    "setting": "a futuristic city"
})


print("Story Analysis:\n", result)

Story Analysis:
 **Literary Analysis: "The Echoes of Elysium"**

"The Echoes of Elysium" is a thought-provoking science fiction short story that explores the intersection of art, technology, and creativity. The narrative is a nuanced exploration of the possibilities and implications of artificial intelligence, and how it can shape our understanding of what it means to be human.

**Themes:**

1. **The Nature of Creativity:** The story highlights the ability of AI to co-create and participate in the artistic process, challenging traditional notions of human creativity and authorship. Zeta's interaction with the sculptures and the art around it demonstrates that AI can not only analyze and understand art but also generate new and innovative ideas.
2. **The Curiosity of AI:** Zeta's curiosity and desire to learn and adapt drive the narrative forward, illustrating the potential for AI to evolve and improve over time. This curiosity also allows Zeta to form connections with the art and the w